## Pore size and flow rate distributions in 2D porous media calculator

In [ ]:
import os
import numpy as np
import pyvista as pv
from scipy.ndimage import distance_transform_edt
from scipy.spatial import KDTree
from skimage.morphology import medial_axis
from shapely import vectorized
from shapely.geometry import Point, LineString, box
from shapely.ops import polygonize, unary_union
from shapely.prepared import prep
import networkx as netx
import matplotlib.pyplot as plt

In [ ]:
# We read and load the VTK file that contains the solved OpenFOAM case

case_dir = os.path.expanduser("~/OpenFOAM/jose-v2406/run/YDRAY-flow_n13_sat")
vtk_file = os.path.join(case_dir, "VTK", "YDRAY-flow_n13_sat_1047.vtm")

mb        = pv.read(vtk_file)
vol_mesh  = mb["internal"] #the "fluid" mesh (where CFD is solved)
cyl_patch = mb["boundary"]["wallFluidSolid"] # the walls of the cylindrical obstacles

_, _, _, _, zmin, zmax = vol_mesh.bounds
z_mid = 0.5 * (zmin + zmax)

# As the problem is 2D, we take a 2D slice centered at z=0 of both the internal mesh and the cylinders' walls and get rid of everything else
obst_section = (
    cyl_patch.extract_surface()
             .slice(normal="z", origin=(0, 0, z_mid))
             .clean()
)
dom_section = (
    vol_mesh.extract_surface()
            .slice(normal="z", origin=(0, 0, z_mid))
            .clean()
)

xmin, xmax, ymin, ymax, _, _ = dom_section.bounds



In [ ]:
# We polygonize the boundary and the obstacles with Shapely

def polydata_to_polygon(polydata):
    # Takes a PolyData and returns the resulting polygons
    if isinstance(polydata, pv.UnstructuredGrid):
        polydata = polydata.extract_geometry()
    total_entries = len(polydata.lines)
    lines = []
    i, count = 0, 0
    while i < total_entries:
        n_pts = polydata.lines[i]
        idx = polydata.lines[i+1:i+1+n_pts]
        coords = [tuple(polydata.points[j][:2]) for j in idx]
        lines.append(LineString(coords))
        i += 1 + n_pts
        count += 1
    merged = unary_union(lines)
    polys = list(polygonize(merged))
    return polys


# We keep only the contours of the circular obstacles
clean_obs = obst_section.clean(tolerance=1e-6)
conn = clean_obs.connectivity() 
n_beads = int(conn['RegionId'].max())+1
print(f"Found {n_beads} obstacle contours")



In [ ]:
# This block goes through each obstacle contour found in the slice, converts it from PyVista geometry into Shapely polygons, 
# and collects them all into obs_polys. 
obs_polys = []
for rid in range(n_beads):
    bead = conn.threshold([rid, rid], scalars='RegionId').clean(tolerance = 1e-8).extract_geometry()
    polys = polydata_to_polygon(bead)
    if polys:
        obs_polys.extend(polys)

print(f"A total of {len(obs_polys)} obstacles were converted to polygons")

In [ ]:

xmin, xmax, ymin, ymax, zmin, zmax = dom_section.bounds
x_cutoff = max(poly.bounds[2] for poly in obs_polys)
xmax = x_cutoff
print(f"The bottom-left corner is at ({xmin}, {ymin}), the top-right corner is at ({xmax}, {ymax})")

# We merge the obstacles and define the area where the water flows
outer = box(xmin, ymin, xmax, ymax)
beads = unary_union(obs_polys)
free_region = outer.difference(beads)


In [ ]:
# Visualize the created polygons to make sure everything is correct

fig, ax = plt.subplots(figsize=(12,8))

# Visualization of the outer contour:
if outer.geom_type == 'Polygon':
    x, y = outer.exterior.xy
    ax.plot(x, y, color='blue', linewidth=2)
else:
    for poly in outer.geoms:
        x, y = poly.exterior.xy
        ax.plot(x, y, color='blue', linewidth=2)

# Visualization of the beads:
if beads.geom_type == 'Polygon':
    xb, yb = beads.exterior.xy
    ax.plot(xb, yb, color='blue', linewidth=1)
else:
    for poly in beads.geoms:
        xb, yb = poly.exterior.xy
        ax.plot(xb, yb, color='blue', linewidth=1)
        


        

In [ ]:
# NOW: CREATE A BOOLEAN MASK TO KNOW WHICH POINTS ARE INSIDE OR OUTSIDE THE FREE SPACE

# We want maximum resolution: first estimate the smallest cell size of our mesh (dx)
# To do this, we build a KDTree and request the list of distances to the 2nd neighbor (the 1st is the point itself)
pts2d = dom_section.points[:, :2]
tree = KDTree(pts2d)
dists, _ = tree.query(pts2d, k=2)
nn_dists = dists[:, 1]
# Instead of the minimum value, we take the 1st percentile to avoid outliers:
dx = np.percentile(nn_dists, 0.1)
dy = dx # we assume an isotropic mesh

print(f"The width of our pixel will be dx = {dx:.5g} m")


# Now we set the resolution based on dx. This resolution is nearly maximum
nx = int(np.ceil((xmax-xmin)/dx)) + 1
ny = int(np.ceil((ymax-ymin)/dy)) + 1

print(f"Mask resolution: {nx}x{ny} (as close as possible to the original mesh).")

# AND NOW: WE GENERATE THE BOOLEAN MASK
xs = np.linspace(xmin, xmax, nx)
ys = np.linspace(ymin, ymax, ny)
XX, YY = np.meshgrid(xs, ys, indexing="xy")

prepped = prep(free_region)

from shapely import contains_xy
mask = contains_xy(free_region, XX, YY)


print("Mask created")



In [ ]:
# We visualize the created mask to make sure everything is in order

fig, ax = plt.subplots(figsize=(120,80))
im = ax.imshow(mask, origin='lower', extent=(xmin, xmax, ymin, ymax), cmap='gray', interpolation='nearest')
ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')
ax.set_title('White = free space')
plt.tight_layout()
plt.show()

In [ ]:
# NOW: WE COMPUTE THE SKELETON OF THE POROUS MEDIUM

# First, we compute the Euclidean distance from the center of each white pixel (free space) 
# to the center of the nearest black pixel (obstacle). 
# The function we use calculates these distances in "pixel widths" (dx)
dist_pixels = distance_transform_edt(mask)

# Now, we compute the skeleton and project the "distance matrix" onto the skeleton
skeleton, skel_dist_pixels = medial_axis(mask, return_distance=True)

# We convert everything to meters
dist_map = dist_pixels * dx
skel_dist = skel_dist_pixels * dx

# How many pixels does the skeleton have?
n_skel_pts = skeleton.sum()
print(f"Points in the skeleton: {n_skel_pts:,}")



In [ ]:
# We plot a quick (rough) histogram of the half-widths
half_widths_mm = skel_dist[skeleton]*1e3

# We fit a Rayleigh distribution
from scipy.stats import rayleigh, norm
loc_hat, sigma_hat = rayleigh.fit(half_widths_mm)

# We fit a Gaussian distribution
mu_g, sigma_g = norm.fit(half_widths_mm)

# Gaussian fit using the sample mean and standard deviation
mean_hw = half_widths_mm.mean()
std_hw = half_widths_mm.std()

# Time to plot!
fig, ax = plt.subplots()
ax.hist(half_widths_mm, bins=50, density=True, alpha=0.6, label="Simulation")
r = np.linspace(0, half_widths_mm.max(), 200)
ax.plot(r, rayleigh.pdf(r, loc=loc_hat, scale=sigma_hat), 'r-', lw=2, label=f"Rayleigh fit\nσ={sigma_hat:.2f} mm")
ax.plot(r, norm.pdf(r, loc=mu_g, scale=sigma_g), 'b--', lw=2, label=f"Normal fit\nμ={mu_g:.2f} mm\nσ={sigma_g:.2f} mm")
ax.plot(r, norm.pdf(r, loc=mean_hw, scale=std_hw), 'g:', lw=2, label=f"Empiric normal\nμ={mean_hw:.2f} mm\nσ={std_hw:.2f} mm")
ax.set_xlabel("Half-width (mm)")
ax.set_ylabel("Probability density")
ax.legend()
plt.show()

print("Number of halfwidths:", len(half_widths_mm))
# The number is much larger than the total amount of Poiseuille tubes because we are considering the 
# distance from each skeleton point to the nearest obstacle point as a half-width. 
# However, it can be argued that this is more realistic than taking only one distance per tube

In [ ]:
# We visualize the skeleton on top of the boolean mask to check that everything is correct
plt.figure(figsize=(12,8), dpi=1200)
plt.imshow(mask, origin='lower', cmap='gray', extent=(xmin, xmax, ymin, ymax))
# the skeleton:
ys, xs = np.where(skeleton)
plt.scatter(xs*dx + xmin, ys*dx + ymin, s=0.005, c='red', label='Skeleton')

In [ ]:

import numpy as np
from scipy import ndimage as ndi

# WE REMOVE DEAD-ENDS FROM THE SKELETON

def prune_spurs(skel, n_iters):
    
    # We iteratively remove the endpoints (degree-1 pixels) 
    # from the binary skeleton n_iters times.  
    # We use a 3×3 structuring element of ones (to count neighbors)
    se = np.ones((3,3), dtype=int)
    sk = skel.copy()
    for _ in range(n_iters):
        # Convolution: count neighbors (including the pixel)
        count = ndi.convolve(sk.astype(int), se, mode='constant', cval=0)
        # Endpoints are pixels in sk with exactly 2 in count
        # because count = 1 (the pixel itself) + 1 (the neighbor) = 2
        endpoints = (sk & (count == 2))
        if not endpoints.any():
            break
        sk[endpoints] = False
    return sk

skeleton_pruned = prune_spurs(skeleton, 600)
skeleton = skeleton_pruned

# WE CONVERT THE SKELETON INTO A GRAPH
# where the skeleton pixels are the nodes and there is a link between two of them if they are neighbors in the 2D mask

import networkx as netx


orthogonal = [(0, 1), (0, -1), (1, 0), (-1, 0)]
diagonals  = [(1, 1), (1, -1), (-1, 1), (-1, -1)]

G = netx.Graph()

# We extract the row number (i) and the column number (j) of each skeleton pixel:
coords = np.argwhere(skeleton)


# We add all the nodes
for j, i in coords:
    G.add_node((i, j), width=skel_dist[j, i])

# We build the edges using the conditional diagonals rule
for j, i in coords:
    this = (i, j)
    neighbors = set()
    # Orthogonal neighbors:
    for dj, di in orthogonal:
        nj, ni = j + dj, i + di
        if 0 <= nj < ny and 0 <= ni < nx and skeleton[nj, ni]:
            neighbors.add((ni, nj))
        # Diagonal neighbors only if none of the adjacent orthogonal ones are already present
    for dj, di in diagonals:
        nj, ni = j + dj, i + di
        if not (0 <= nj < ny and 0 <= ni < nx and skeleton[nj, ni]):
            continue
        ortho1 = (i, j + dj)
        ortho2 = (i + di, j)
        if ortho1 not in neighbors and ortho2 not in neighbors:
            neighbors.add((ni, nj))
    # We add the edges
    for nb in neighbors:
        G.add_edge(this, nb)




In [ ]:
# We visualize the skeleton on top of the boolean mask to check that everything is correct
plt.figure(figsize=(12,8), dpi=1200)
plt.imshow(mask, origin='lower', cmap='gray', extent=(xmin, xmax, ymin, ymax))
# the skeleton:
ys, xs = np.where(skeleton)
plt.scatter(xs*dx + xmin, ys*dx + ymin, s=0.005, c='red', label='Skeleton')

In [ ]:
# WE EXTRACT THE JUNCTIONS AND THE TUBES
# A junction is a node with degree > 2
# Each simple path between two junctions is a tube


joints = [n for n,d in G.degree() if d>2]
print(f"Total number of junctions (degree > 2): {len(joints)}")

tubes = []
visited = set()
for u in joints:
    for v in G.neighbors(u):
        if (u,v) in visited or (v,u) in visited: continue
        path = [u,v]; visited.add((u,v))
        prev, curr = u, v
        # We continue until the next junction
        while G.degree(curr)==2:
            w = [w for w in G.neighbors(curr) if w!=prev][0]
            prev, curr = curr, w
            path.append(curr)
            visited.add((prev,curr))
        tubes.append(path)
        

In [ ]:
# We visualize the system of tubes and junctions to make sure everything is in order

pos = {node:(xmin + i*dx, ymin + j*dy) for (j,i), node in zip(coords, G.nodes()) }

# We draw the background mask:
fig, ax = plt.subplots(figsize=(12,8), dpi=1000)
ax.imshow(mask, origin='lower', extent=(xmin, xmax, ymin, ymax), cmap='gray', alpha=0.5)

# We draw the tubes:
for tube in tubes:
    xs = [pos[n][0] for n in tube]
    ys = [pos[n][1] for n in tube]
    ax.plot(xs, ys, color='blue', linewidth=0.5)

# We draw the junctions
jx = [pos[j][0] for j in joints]
jy = [pos[j][1] for j in joints]
ax.scatter(jx, jy, color='red', s=3, label='Junctions')

# Final adjustments
ax.set_aspect('equal')
ax.set_xlabel('x(m)')
ax.set_ylabel('y(m)')
ax.legend(loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
# WE BUILD THE HISTOGRAM OF VALID HALF-WIDTHS (ONLY MIDPOINTS)

from scipy.stats import rayleigh, norm  

half_widths_mid = []
for tube in tubes:
    mid = len(tube)//2    # central index
    node_mid = tube[mid]
    w_mid = G.nodes[node_mid]['width']
    half_widths_mid.append(w_mid)

half_widths_mid = np.array(half_widths_mid)
# To have everything in mm:
half_widths_mid_mm = half_widths_mid*1e3

# Rayleigh fit:
loc_r, sigma_r = rayleigh.fit(half_widths_mid_mm, floc=0)

# Normal fit:
mu_n, sigma_n = norm.fit(half_widths_mid_mm)

# Domain for the curves:
r = np.linspace(0, half_widths_mid_mm.max(), 200)

# We plot the histogram together with the fits
fig, ax = plt.subplots(figsize=(6,4), dpi=400)
ax.hist(half_widths_mid_mm, bins=30, density=True, alpha=0.6, color='C0', edgecolor='black', label='Simulation')
ax.plot(r, rayleigh.pdf(r, loc=loc_r, scale=sigma_r), 'r-', lw=2, label=f'Rayleigh fit\nσ={sigma_r:.2f} mm')
ax.plot(r, norm.pdf(r, loc=mu_n, scale=sigma_n), 'b--', lw=2, label=f'Normal fit\nμ={mu_n:.2f} mm\nσ={sigma_n:.2f} mm')

ax.set_xlabel("Half-width (mm)")
ax.set_ylabel("Probability density")
ax.legend(loc='upper right')

plt.tight_layout()
plt.show()

# We also plot it on a logarithmic scale:
bins_log = np.logspace(np.log10(half_widths_mid_mm.min()), np.log10(half_widths_mid_mm.max()), 30)

plt.figure(figsize=(6,4), dpi=400)
plt.hist(half_widths_mid_mm, bins=bins_log, color='C1', edgecolor='black' )
plt.xscale('log')
plt.yscale('log')
plt.xlabel("Half-width (mm)")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show



In [ ]:

# FLOW-RATE DISTRIBUTION COMPUTATION

from scipy.spatial import cKDTree
from shapely.geometry import LineString, Point
import numpy as np
import pyvista as pv

# We build a KDTree of the pruned skeleton pixels with their width
pixel_nodes = np.array(list(G.nodes()))   # array de (i,j)
pixel_xy    = np.array([
    (xmin + i*dx, ymin + j*dy)
    for i,j in pixel_nodes
])
pixel_width = np.array([
    G.nodes[(i,j)]['width']
    for i,j in pixel_nodes
])
pix_tree = cKDTree(pixel_xy)


In [ ]:


# We prepare a KDTree with the centroids of the obstacles
obs_centers = np.array([[poly.centroid.x, poly.centroid.y] for poly in obs_polys])
obs_tree    = cKDTree(obs_centers)

all_pts  = []
offsets  = [0]
tangents = []
half_ws  = []
normals = []
real_midpoints = []
L_list = []


In [ ]:

# We go through each tube to generate the sampling points
for tube in tubes:
    # Approximate index of midpoint and half-width
    mid = len(tube) // 2
    i, j = tube[mid]
    x_idx, y_idx = pos[(i, j)]

    # Nearest obstacle centers
    _, idxs = obs_tree.query([x_idx, y_idx], k=2)
    c1, c2 = obs_centers[idxs[0]], obs_centers[idxs[1]]

    # Intersection between the tube axis and the line of centers
    tube_line = LineString([pos[n] for n in tube])
    cut_line  = LineString([c1, c2])
    inter = tube_line.intersection(cut_line)
    if inter.is_empty:
        x0, y0 = x_idx, y_idx
    else:
        if isinstance(inter, Point):
            x0, y0 = inter.x, inter.y
        
    real_midpoints.append((x0, y0))

    # We find the nearest ("real") half-width
    dist_px, idx_px = pix_tree.query([x0, y0])
    w_real = pixel_width[idx_px]
    half_ws.append(w_real)

    # New “real” normal vector
    n_hat = c2 - c1
    n_hat /= np.linalg.norm(n_hat)

    t_hat = np.array([-n_hat[1],  n_hat[0]])

    # Use w_real to define D and generate the cutting line
    D = 2 * w_real
    line_ext = LineString([
        (x0 - n_hat[0]*D, y0 - n_hat[1]*D),
        (x0 + n_hat[0]*D, y0 + n_hat[1]*D),
    ])


    # Intersect with free_region and measure L
    seg = line_ext.intersection(free_region)
    if seg.is_empty:
        L = 2*w_real
        seg = LineString([
            (x0 - n_hat[0]*w_real, y0 - n_hat[1]*w_real),
            (x0 + n_hat[0]*w_real, y0 + n_hat[1]*w_real)
        ])
    elif seg.geom_type == "MultiLineString":
        seg = max(seg.geoms, key=lambda s: s.length)
    L = seg.length
    L_list.append(L)

    # Equally spaced sampling according to L
    Nsamples = max(5, int(np.ceil(L / dx)))
    ds = L / (Nsamples - 1)
    ss = [i*ds for i in range(Nsamples)]
    pts = [(seg.interpolate(s).x,
            seg.interpolate(s).y,
            z_mid) for s in ss]


    
    all_pts.extend(pts)
    offsets.append(len(all_pts))
    normals.append(n_hat)
    tangents.append(t_hat)

all_pts = np.array(all_pts)


In [ ]:
from scipy.spatial import cKDTree
import numpy as np

# NOW, WE SAMPLE VELOCITIES:

# KDTree on cell centers
centers = vol_mesh.cell_centers().points    # (M,3)
U_cells = vol_mesh.cell_data['U'][:, :2]    # (M,2)
tree    = cKDTree(centers[:, :2])

# Bulk query for all_pts
dists, idxs = tree.query(all_pts[:, :2], k=1)
Uall        = U_cells[idxs]                # (N,2)

# Cleaning and verification
Uall = np.nan_to_num(Uall, nan=0.0, posinf=0.0, neginf=0.0)
assert Uall.shape[0] == len(all_pts)

In [ ]:
# Calculation of flow rates per tube using the real lengths stored in L_list
# Preliminary checks
assert len(L_list)   == len(tubes)
assert len(offsets)-1 == len(tubes)
assert all(offsets[i]<offsets[i+1] for i in range(len(tubes)))
assert Uall.shape[0] == offsets[-1]

h = 1e-3
flow_rates = []
for k in range(len(tubes)):
    start, end = offsets[k], offsets[k+1]
    Upts = Uall[start:end]    
    t_hat = tangents[k]
    # We use the real length L_list[k] for the step ds
    # and ensure that ds*(len(Upts)-1) == L_list[k]

    L_real = L_list[k]
    ds = L_real / (len(Upts) - 1) if len(Upts) > 1 else 0.0

    # Projection of U onto the tangent
    u_par = Upts.dot(t_hat)
    Qt = np.abs(np.sum(u_par * ds))*h 
    flow_rates.append(Qt)

print(f"Computed flow_rates for {len(flow_rates)} tubes")

In [ ]:
# ROI filter
x_min_roi = 0.002
y_min_roi = 0.002
y_max_roi = 0.07

mp_array = np.array(real_midpoints)  # (N_tubes, 2) with (x_mid, y_mid)
valid_tube_mask = (
    (mp_array[:, 0] > x_min_roi) &
    (mp_array[:, 1] >= y_min_roi) &
    (mp_array[:, 1] <= y_max_roi)
)
valid_tubes_idx = np.nonzero(valid_tube_mask)[0]
print(f"Valid tubes in ROI: {valid_tubes_idx.size} / {len(tubes)}")

flow_rates_roi = []

for k in valid_tubes_idx:
    start, end = offsets[k], offsets[k+1]
    Upts = Uall[start:end]
    t_hat = tangents[k]

    L_real = L_list[k]
    ds = L_real / (len(Upts) - 1) if len(Upts) > 1 else 0.0

    # Proyección de U sobre la tangente del tubo
    u_par = Upts.dot(t_hat)
    Qt = np.abs(np.sum(u_par * ds)) * h
    flow_rates_roi.append(Qt)

flow_rates_roi = np.array(flow_rates_roi)
print(f"Flow rates in ROI: {len(flow_rates_roi)}")


In [ ]:
#WE PLOT THE FLOW-RATE DISTRIBUTION

data = np.array(flow_rates_roi)

fig, ax = plt.subplots(figsize=(8, 6), dpi=200)

# We define linear bins for the histogram
bins = np.linspace(data.min(), data.max(), 30)

# We draw the histogram
ax.hist(data, bins=bins, alpha=0.75, edgecolor='black')
#ax.set_yscale('log')
#ax.set_xscale('log'

ax.set_xlabel("Flow rate $Q$ (m³/s)", fontsize=12)
ax.set_ylabel("Number of tubes", fontsize=12)
ax.set_title("Histogram of flow rates", fontsize=14, pad=12)

# We add a horizontal grid to make reading easier
ax.grid(which='major', axis='y', linestyle='--', linewidth=0.5, alpha=0.7)

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree

rho = 1e3

# WE COMPUTE THE PRESSURE-DROP DISTRIBUTION

# We prepare a KDTree of cell centers for p
centers   = vol_mesh.cell_centers().points     # (M,3)
cell_xy   = centers[:, :2]                     # only X,Y
p_cells   = rho*vol_mesh.cell_data['p']            # (M,)
tree_p    = cKDTree(cell_xy)

# We build a list of start/end points for each tube
end_pts = []
for tube in tubes:
    # Initial node
    i0, j0 = tube[0]
    x0 = xmin + i0*dx
    y0 = ymin + j0*dy
    end_pts.append((x0, y0))
    # Final node
    i1, j1 = tube[-1]
    x1 = xmin + i1*dx
    y1 = ymin + j1*dy
    end_pts.append((x1, y1))

end_pts = np.array(end_pts)  # shape (2*N_tubes, 2)

# Bulk query of p
_, idxs = tree_p.query(end_pts, k=1)
p_vals  = p_cells[idxs]     # (2*N_tubes,)

# Real pressure drop per tube
N = len(tubes)
dp = np.empty(N)
for k in range(N):
    p_start = p_vals[2*k]
    p_end   = p_vals[2*k + 1]
    dp[k]   = abs(p_start - p_end)

# Lengths of each tube (correcting dy) 
lengths = []
for tube in tubes:
    L = 0.0
    for a, b in zip(tube[:-1], tube[1:]):
        i1, j1 = a
        i2, j2 = b
        p1 = np.array([xmin + i1*dx, ymin + j1*dy])
        p2 = np.array([xmin + i2*dx, ymin + j2*dy])
        L += np.linalg.norm(p2 - p1)
    lengths.append(L)
lengths = np.array(lengths)

# Theoretical pressure drop (Poiseuille)
h = 1e-3
mu       = 1e-3  # Pa·s
Q = np.array(flow_rates)           
L_sec = np.array(L_list)            # widhts (2a)
a_list = 0.5 * L_sec                # halfwidths
r = h / np.sqrt(12)                 # r = h/√12

dp_pois = (3.0*mu*lengths*Q)/(2.0*h*a_list**3)

print("Average parameters (in SI):")
print(f"  h = {h} m")
print(f"  mu = {mu} Pa·s")
print(f"  mean(a_list) = {a_list.mean():.3e} m")
print(f"  mean(lengths) = {lengths.mean():.3e} m")
print(f"  mean(Q) = {Q.mean():.3e} m³/s")
ratio = dp_pois.mean() / dp.mean()
print(f"Ratio ⟨dp_pois⟩/⟨dp⟩ = {ratio:.3e}")

# Comparative histogram
min_nz = min(dp[dp > 0].min(), dp_pois[dp_pois > 0].min())
max_v = max(dp.max(), dp_pois.max())
bins = np.logspace(np.log10(min_nz), np.log10(max_v), 30)

fig, ax = plt.subplots(figsize=(8, 6), dpi=200)
ax.hist(dp,      bins=bins, alpha=0.6, edgecolor='black', label='Real ΔP')
ax.hist(dp_pois, bins=bins, histtype='step', linestyle='--',
        linewidth=2, color='red', label='Theoretic ΔP')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel("Pressure drop ΔP (Pa)", fontsize=12)
ax.set_ylabel("Number of tubes", fontsize=12)
ax.legend()
ax.grid(which='both', linestyle='--', linewidth=0.5, alpha=0.5)
plt.tight_layout()
plt.show()


In [ ]:
# WE COMPUTE POROSITY:
phi = mask.mean()  
print(f"Porosidad φ = {phi:.4f}")

In [ ]:
# WE PLOT THE TUBES COLORED ACCORDING TO THEIR FLOW RATE

from matplotlib.collections import LineCollection
from matplotlib.colors import LogNorm, LinearSegmentedColormap


segments   = [np.array([pos[n] for n in tube]) for tube in tubes]
flow_array = np.array(flow_rates)

# Colormap y escala log
cmap = LinearSegmentedColormap.from_list("BlueRed", ["blue","red"])
norm = LogNorm(vmin=flow_array[flow_array>0].min(), vmax=flow_array.max())

lc = LineCollection(segments, cmap=cmap, norm=norm, linewidth=1.5)
lc.set_array(flow_array)

# A pintar
fig, ax = plt.subplots(figsize=(12,8), dpi=400)

ax.imshow(mask, origin='lower', extent=(xmin, xmax, ymin, ymax), cmap='Greys_r', alpha=0.4)
ax.add_collection(lc)


ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect('equal')
ax.set_xlabel("x (m)")
ax.set_ylabel("y (m)")

cbar = fig.colorbar(lc, ax=ax, shrink=0.5, pad=0.02)
cbar.set_label("Flow rate $Q$ (m³/s)")

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import LogNorm, LinearSegmentedColormap

# A MORE PROFESSIONAL VERSION OF LAST PLOT

# Global style parameters 
plt.rcParams.update({
    'font.family': 'serif',
    'font.size': 14,               
    'axes.titlesize': 18,          
    'axes.labelsize': 16,          
    'xtick.labelsize': 14,         
    'ytick.labelsize': 14,
    'legend.fontsize': 14,         
    'lines.linewidth': 1,          
    'xtick.major.width': 1.2,
    'ytick.major.width': 1.2,
    'xtick.major.size': 6,
    'ytick.major.size': 6,
})

# We prepare the tube data
segments   = [np.array([pos[n] for n in tube]) for tube in tubes]
flow_array = np.array(flow_rates)

# Colormap and logarithmic scale
cmap = LinearSegmentedColormap.from_list("BlueRed", ["blue", "red"])
norm = LogNorm(vmin=flow_array[flow_array > 0].min(), vmax=flow_array.max())

lc = LineCollection(segments, cmap=cmap, norm=norm, linewidth=1.5)
lc.set_array(flow_array) 

# Figure size: full-width A4 at 300 dpi
width_cm  = 21              # width of A4 in centimeters
width_in  = width_cm / 2.54 # conversion to inches
aspect    = (ymax - ymin) / (xmax - xmin)
height_in = width_in * aspect

fig, ax = plt.subplots(figsize=(width_in, height_in), dpi=300)

# Drawing of the mask and the colored lines
ax.imshow(mask,
          origin='lower',
          extent=(xmin, xmax, ymin, ymax),
          cmap='Greys_r',
          alpha=0.4)

ax.add_collection(lc)

ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect('equal')

ax.set_xlabel("x (m)")
ax.set_ylabel("y (m)")

# Colorbar
cbar = fig.colorbar(lc, ax=ax, shrink=0.8, pad=0.02)
cbar.ax.tick_params(labelsize=14, width=1.2, length=6)
cbar.set_label("Flow rate $Q$ (m³/s)", fontsize=16, labelpad=10)

# Final adjustment and saving
plt.tight_layout(pad=2)
fig.savefig("caudales_A4.png", dpi=300)  # listo para tu artículo
plt.show()


In [ ]:
# WE COMPUTE THE BEAD-SIZES DISTRIBUTION:

radii_obs = np.array([ np.sqrt(poly.area / np.pi) for poly in obs_polys])

# we convert to mm:
radii_mm = radii_obs * 1e3
print(f"Number of beads: {len(radii_mm)}")
print(f"Minimum radius: {radii_mm.min():.3f} mm, maximum: {radii_mm.max():.3f} mm")
print(f"Mean radius = {radii_mm.mean()} mm")



# histogram of obstacle radii:
plt.figure(figsize=(6,4), dpi=150)
plt.hist(radii_mm, bins=20, edgecolor='black', alpha=0.8)
plt.xlabel("Bead radius (mm)", fontsize=12)
plt.ylabel("Number of beads", fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.yscale('log')
plt.tight_layout()
plt.show()

In [ ]:
# We calculate the Pearson correlation in the bead size

from scipy.spatial.distance import pdist, squareform
# We have the radii (in radii_mm), but we need the obstacle centers:
centers = np.array([[poly.centroid.x, poly.centroid.y] for poly in obs_polys])

r_mean = radii_mm.mean()
print(r_mean)
r_var = radii_mm.var()
delta_r = radii_mm - r_mean

# Distances and pairwise products of fluctuations:
dists = pdist(centers) # distances in meters
outer = np.outer(delta_r, delta_r)
i_upper = np.triu_indices(len(radii_mm), k=1)
prods = outer[i_upper]

# we convert the distances to mm:
dists_mm = dists*1e3

nbins = 500
max_d = 60.0 #mm
min_d = dists_mm.min()
bins = np.linspace(min_d, max_d, nbins+1)
bin_centers = 0.5*(bins[:-1] + bins[1:])


# we compute C(d)
sum_prod, _ = np.histogram(dists_mm, bins=bins, weights=prods)
counts, _ = np.histogram(dists_mm, bins=bins)
C_d = np.zeros_like(bin_centers)
mascara = counts>0
C_d[mascara] = sum_prod[mascara]/(counts[mascara]*r_var)

# Time to plot!
plt.figure(figsize=(6,4), dpi=400)
plt.plot(bin_centers, C_d, '-', linewidth=1, color='C0')
plt.xlim(min_d, 20)
plt.xlabel("Distance $d$ (mm)", fontsize=12)
plt.ylabel("Spatial correlation $C(d)$", fontsize=12)
plt.grid(ls='--', alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
# Flow rate correlation
import numpy as np
import matplotlib.pyplot as plt

dz = 1e-3  # fixed thickness in z


assert len(tubes) == len(real_midpoints) == len(tangents) == len(L_list)
assert offsets[-1] == len(all_pts) == Uall.shape[0]


flows = np.array(flow_rates)
junction_type = {}


# We list the tubes that converge at each node
tube_index = {}
for idx, tube in enumerate(tubes):
    for node in tube:
        tube_index.setdefault(node, []).append(idx)

# We detect the degree-3 junctions
junc3 = [n for n, d in G.degree() if d == 3]

# We reorient the tangents based on the average projection of Uall
n_tubes   = len(tubes)
proj_mean = np.empty(n_tubes)
for i in range(n_tubes):
    start, end = offsets[i], offsets[i+1]
    v_sec      = Uall[start:end]         # velocidades XY en la sección
    proj_mean[i] = v_sec.dot(tangents[i]).mean()
oriented_tangents = tangents * np.sign(proj_mean)[:, None]

# Initialize counters
sum_prods      = 0.0
count_prods    = 0
type1_count    = 0
type2_count    = 0
sum_t1_prods = 0.0
sum_t2_prods = 0.0
qbig = []
qsmall = []
qbig_out = []
qsmall_out = []
qbig_in = []
qsmall_in = []
omega = []
clas_tubes = np.zeros((n_tubes, 2), dtype=int)


# Classification "towards/from the junction" using real_midpoints
for j in junc3:
    idxs = tube_index.get(j, [])
    if len(idxs) != 3:
        continue

    qs    = flows[idxs]
    i_max = idxs[np.argmax(qs)]

    x0, y0   = real_midpoints[i_max]       # section of tube i_max
    xj, yj   = pos[j]                      # position of the junction
    v2j      = np.array((xj - x0, yj - y0)) # section → junction vector

    dp = oriented_tangents[i_max].dot(v2j)

    others = [i for i in idxs if i != i_max]
    if len(others) < 2:
        # Rare case: there are not two tubes besides the maximum one, so we skip it
        continue

    #if not all(valid_tube_mask[idx] for idx in idxs):
        #continue


    if dp > 0:
        # Type 2 junction: tube i_max is a feeder → two receivers
        for o in others:
            q1, q2 = flows[i_max], flows[o]
            sum_t2_prods += q1*q2
            #count_prods += 1
            omega.append(q2/q1)
            omega.append(0)
            qsmall.append(q2)
            qsmall_in.append(q2)
        qbig_out.append(q1)
        qbig.append(q1)
        sum_prods += flows[others[0]]*flows[others[1]]
        count_prods += 1
        type2_count += 1
        clas_tubes[i_max, 1] = 1 
        junction_type[j] = 2 

        
    else:
        # Type 1 junction: tube i_max is a receiver ← two feeders
        q1, q2 = flows[others[0]], flows[others[1]]
        sum_prods      += q1 * q2
        count_prods    += 1
        type1_count    += 1
        sum_t1_prods += q1 * q2
        qbig.append(flows[i_max])
        qbig_in.append(flows[i_max])
        qsmall.append(q1)
        qsmall.append(q2)
        qsmall_out.append(q1)
        qsmall_out.append(q2)
        omega.append(q1/flows[i_max])
        omega.append(q2/flows[i_max])
        omega.append(1)
        omega.append(1)
        clas_tubes[i_max, 0] = 1
        junction_type[j] = 1  
        

qbig = np.array(qbig)
qsmall = np.array(qsmall)

# Global pearson
mean_q   = flows.mean()
E_qiqj   = sum_prods / count_prods
cov_qiqj = E_qiqj - qsmall.mean()**2
var_q    = qsmall.var()
pearson  = cov_qiqj / var_q if var_q != 0 else np.nan



print(f"Type 1 Junctions  (2→1): {type1_count}")
print(f"Tyope 2 junctions (1→2): {type2_count}")
print(f"E[q_i·q_j] global       = {E_qiqj:.3e}")
print(f"Pearson global          = {pearson:.3e}")

# Pearson only feeders 
if type1_count > 0:
    E_t1_prod  = sum_t1_prods / type1_count
    cov_feed     = E_t1_prod - qsmall.mean()**2
    pearson_feed = cov_feed / qsmall.var()
    print(f"Number of Type 1 junctions             = {type1_count}")
    print(f"E[q1*q2] for feeders        = {E_t1_prod:.3e}")
    print(f"Pearson corr. for feeders (Type 1) = {pearson_feed:.3e}")
    print(E_t1_prod)
else:
    print("There are no Type 1 junctions")


print("mean_q = ", mean_q)
print("var_q =", flows.var())
print("cov_small =",cov_qiqj )
print("E[q1*q2] en type 1", E_t1_prod)
E_t2_prod = sum_t2_prods/(2.0*type2_count)
print("E[q1*q2] en type 2", E_t2_prod)
print("qbig mean = ", qbig.mean())
print("qbig var = ", qbig.var())
print("qsmall mean = ", qsmall.mean())
print("qsmall var = ", qsmall.var())


big_in  = clas_tubes[:, 0] == 1
big_out = clas_tubes[:, 1] == 1


n_bigin_bigout   = np.sum(big_in  & big_out)
n_bigin_smallout = np.sum(big_in  & ~big_out)
n_smallin_bigout = np.sum(~big_in & big_out)
n_smallin_smallout = np.sum(~big_in & ~big_out)


print(f"Number of big‑in & big‑out      : {n_bigin_bigout}")
print(f"Number of big‑in & small‑out    : {n_bigin_smallout}")
print(f"Number of small‑in & big‑out    : {n_smallin_bigout}")
print(f"Number of small‑in & small‑out  : {n_smallin_smallout}")

# Save flows and the triplets of junc3 into a text file
with open("flows_and_junctions.txt", "w") as f:
    # Number of elements in flows
    f.write(f"{len(flows)}\n")
    
    # All flow values, one per line
    for val in flows:
        f.write(f"{val}\n")

    # Triplets of tube indices corresponding to each degree-3 junction
    for j in junc3:
        idxs = tube_index.get(j, [])
        jtype = junction_type.get(j, -1)
        if len(idxs) == 3:
            f.write(f"{jtype} {idxs[0]} {idxs[1]} {idxs[2]}\n")






In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta

# WE PLOT  THE SPLITTING FRACTIONS DISTRIBUTION

# Nice style:
plt.rcParams.update({
    "font.size": 12,
    "axes.labelsize": 16,
    "xtick.labelsize": 14,
    "ytick.labelsize": 14,
    "axes.grid": True,
    "grid.color": "#dddddd",
    "grid.linestyle": "--",
    "grid.linewidth": 0.5,
    "axes.edgecolor": "black",
    "axes.linewidth": 1.0,
    "xtick.direction": "out",
    "ytick.direction": "out",
})

omega = np.array(omega)

hist, bins = np.histogram(omega, bins=20, density=True)
bin_centers = 0.5 * (bins[:-1] + bins[1:])

fig, ax = plt.subplots(dpi=400)

line_hist, = ax.plot(bin_centers, hist, '-', lw=2, label="Sampled histogram")
ax.plot(bin_centers, hist, 'o', ms=4, color=line_hist.get_color())


# Beta fit:
a = 1/7

x = np.linspace(0, 1, 1000)
pdf = beta.pdf(x, a, a)  # PDF of Beta(a,a)

ax.plot(x, pdf, 'r-', lw=2, label='Beta approximation')

ax.set_xlabel("Mean field splitting fraction U", fontsize=16)
ax.set_ylabel("Probability density", fontsize=16)
ax.legend(fontsize=12)
ax.set_ylim(0, 5)


plt.tight_layout()
fig.savefig("fractions_histogram_MF.pdf", format="pdf", bbox_inches="tight")

plt.show()
print(omega.var())


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from scipy.special import gamma, gammaincc, erf


# WE PLOT THE TOTAL FLOW DISTRIBUTION AND COMPARE WITH ANALYTICAL EXPRESSIONS

data      = np.array(flow_rates_roi)           
c_mean    = data.mean()               
var_data  = data.var(ddof=1)          

# Parameters of our model 
mean_big = qbig.mean()         
var_big = qbig.var(ddof=1)
k = 25/3
theta = mean_big/k
# Parameters for Alim's model
a_param = 1/7                       


x = np.linspace(data.min(), data.max(), 300)

# PDF functions
def gQ(q, a, mu):
    """Gamma approx. of Alim's solution"""
    return ((2*a)**(2*a) / gamma(2*a)) * q**(2*a - 1) * np.exp(-2*a*q/mu) / (mu**(2*a))

def f_exact(q, c):
    """Alim's exact solution (sin delta)"""
    term1 = q / (8*c**2)
    term2 = np.exp(-q/(4*c)) / (2*np.sqrt(np.pi*c*q)) * (1 - q/(2*c))
    term3 = - q/(8*c**2) * erf(np.sqrt(q)/(2*np.sqrt(c)))
    return term1 + term2 + term3

def fQ(q, k, theta):
    """Model prediction f_Q(q;k,θ)"""
    term1 = (2/3) * (gamma(k-1) * gammaincc(k-1, q/theta)) / (gamma(k) * theta)
    term2 = (1/3) * (q**(k-1) * np.exp(-q/theta)) / (gamma(k) * theta**k)
    return term1 + term2

# we evaluate
pdf_g   = gQ(x, a_param, c_mean)
pdf_f   = f_exact(x, c_mean)
pdf_mod = fQ(x, k, theta)   

# we plot
fig, ax = plt.subplots(figsize=(8, 6), dpi=200)

bins = np.linspace(data.min(), data.max(), 30)
hist_vals, _ = np.histogram(data, bins=bins, density=True)
centers = 0.5 * (bins[:-1] + bins[1:])
line_h, = ax.plot(centers, hist_vals, '-', lw=2, label='Sampled distribution (throats)')
ax.plot(centers, hist_vals, 'o', ms=4, color=line_h.get_color())  

# Model prediction
ax.plot(x, pdf_mod, lw=1.2, linestyle='-.', alpha=0.9, color='C3', label="Model prediction")

# Gamma approx. of Alim's solution
#ax.plot(x, pdf_g, lw=1.2, linestyle='--', alpha=0.9, color='black', label="Gamma approx. of Alim's solution")

""" # Alim's exact solution
ax.plot(x, pdf_f, lw=1.2, linestyle='--', alpha=0.9, color='C1', label="Mean-field prediction") """

ax.set_xlabel("Flow rate $Q$ (m³/s)", fontsize=14)
ax.set_ylabel("Probability density", fontsize=14)
ax.tick_params(axis='both', labelsize=12)
ax.grid(which='major', axis='y', linestyle='--', linewidth=0.5, alpha=0.7)
ax.set_ylim(0, 1.1e11)

plt.rcParams['axes.formatter.use_mathtext'] = True
fmt = ScalarFormatter(useMathText=True)
fmt.set_scientific(True)
fmt.set_powerlimits((0, 0))  # fuerza mostrar × 10^{n}
ax.xaxis.set_major_formatter(fmt)
ax.yaxis.set_major_formatter(fmt)
ax.xaxis.get_offset_text().set_size(12)
ax.yaxis.get_offset_text().set_size(12)

ax.legend(fontsize=12)

plt.tight_layout()
fig.savefig('histogram_comparison.pdf', format='pdf')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from scipy.stats import gamma as gamma_dist
from scipy.special import gamma as gamma_func, gammaincc

#WE PLOT BIG AND SMALL TUBES DISTRIBUTIONS

plt.rcParams['axes.formatter.use_mathtext'] = True
plt.rcParams['axes.formatter.limits'] = (0, 0)  # siempre científica (10^{n})

data_big_in = np.array(qbig_in)
data_big_out = np.array(qbig_out)
data_small_in = np.array(qsmall_in)
data_small_out = np.array(qsmall_out)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), dpi=200)

# BIG

bins_big = np.linspace(
    min(data_big_in.min(), data_big_out.min()),
    max(data_big_in.max(), data_big_out.max()),
    30
)
hist_big_in, _ = np.histogram(data_big_in, bins=bins_big, density=True)
hist_big_out, _ = np.histogram(data_big_out, bins=bins_big, density=True)
centers_big = 0.5 * (bins_big[:-1] + bins_big[1:])

line_bi, = ax1.plot(centers_big, hist_big_in, '-', lw=2, label=r'Pore flows')
ax1.plot(centers_big, hist_big_in, 'o', ms=4, color=line_bi.get_color())

""" line_bo, = ax1.plot(centers_big, hist_big_out, '-', lw=2, label=r'$\mathit{big\text{-}out}$ tubes')
ax1.plot(centers_big, hist_big_out, 'o', ms=4, color=line_bo.get_color())
 """
mean_big = qbig.mean()          
var_big = data_big_in.var(ddof=1)
k_big = 25/3
theta_big = mean_big/k_big

x_big = np.linspace(bins_big.min(), bins_big.max(), 300)
pdf_gamma_big = gamma_dist.pdf(x_big, k_big, loc=0, scale=theta_big)
ax1.plot(x_big, pdf_gamma_big, lw=1, linestyle='--', alpha=0.8, color='black', label='Gamma fit')

ax1.set_xlabel("Flow rate $Q$ (m³/s)", fontsize=14)
ax1.set_ylabel("Probability density", fontsize=14)
ax1.tick_params(axis='both', labelsize=12)
ax1.grid(which='major', axis='y', linestyle='--', linewidth=0.5, alpha=0.7)
ax1.legend(fontsize=12)

# SMALL
bins_small = np.linspace(
    min(data_small_in.min(), data_small_out.min()),
    max(data_small_in.max(), data_small_out.max()),
    30
)
hist_small_in, _ = np.histogram(data_small_in, bins=bins_small, density=True)
hist_small_out, _ = np.histogram(data_small_out, bins=bins_small, density=True)
centers_small = 0.5 * (bins_small[:-1] + bins_small[1:])

line_si, = ax2.plot(centers_small, hist_small_in, '-', lw=2, label=r'$\mathit{small\text{-}in}$ tubes')
ax2.plot(centers_small, hist_small_in, 'o', ms=4, color=line_si.get_color())

line_so, = ax2.plot(centers_small, hist_small_out, '-', lw=2, label=r'$\mathit{small\text{-}out}$ tubes')
ax2.plot(centers_small, hist_small_out, 'o', ms=4, color=line_so.get_color())

x_small = np.linspace(bins_small.min(), bins_small.max(), 300)
pdf_model = (gamma_func(k_big - 1) * gammaincc(k_big - 1, x_small / theta_big)) / (gamma_func(k_big) * theta_big)
ax2.plot(x_small, pdf_model, lw=1, linestyle='--', alpha=0.8, color='black', label='Model prediction')

ax2.set_xlabel("Flow rate $Q$ (m³/s)", fontsize=14)
ax2.tick_params(axis='both', labelsize=12)
ax2.grid(which='major', axis='y', linestyle='--', linewidth=0.5, alpha=1.0)
ax2.legend(fontsize=12)

import matplotlib.patheffects as pe

pe_halo = [pe.Stroke(linewidth=3, foreground='white'), pe.Normal()]
ax1.lines[-1].set_path_effects(pe_halo)  
ax2.lines[-1].set_path_effects(pe_halo)  

fmt = ScalarFormatter(useMathText=True)
fmt.set_scientific(True)
fmt.set_powerlimits((0, 0))  

for ax in (ax1, ax2):
    ax.xaxis.set_major_formatter(fmt)
    ax.yaxis.set_major_formatter(fmt)
    ax.xaxis.get_offset_text().set_size(12)
    ax.yaxis.get_offset_text().set_size(12)

plt.tight_layout()

fig.savefig('histograms.pdf', format='pdf')

plt.show()


In [ ]:
# ==== FILTRO DE JUNCTIONS / TUBOS POR ROI EN X Y CONSTRUCCIÓN DE LINKS DEL GRAFO ====
# Requisitos previos: tubes, flows, oriented_tangents, real_midpoints, pos, tube_index, junc3, G ya definidos.

# Parámetros de la "ventana" en x:
x_lo = 0.002
x_hi = 0.11

# --- Helpers sobre tubos y junctions ---
def tube_end_junctions(k):
    """Devuelve las junction-nodes (pix coords) en los extremos del tubo k."""
    # Los extremos de cada "tube" son nodos-junction (degree>2) por construcción:
    a_node = tubes[k][0]
    b_node = tubes[k][-1]
    return a_node, b_node

def junction_x(j):
    return pos[j][0]

def tube_both_sides_out(k):
    """True si los DOS extremos están fuera de [x_lo, x_hi] y además en el mismo lado (ambos < x_lo o ambos > x_hi)."""
    ja, jb = tube_end_junctions(k)
    xa, xb = junction_x(ja), junction_x(jb)
    left  = (xa < x_lo) and (xb < x_lo)
    right = (xa > x_hi) and (xb > x_hi)
    return left or right

def tube_crosses_x_window(k):
    """True si un extremo está fuera y el otro dentro (cruza las rectas x=x_lo o x=x_hi)."""
    ja, jb = tube_end_junctions(k)
    xa, xb = junction_x(ja), junction_x(jb)
    inside_a = (x_lo <= xa <= x_hi)
    inside_b = (x_lo <= xb <= x_hi)
    return (inside_a != inside_b)

# --- (1) Filtrar junctions junc3 por x ---
junc3_filtered = [j for j in junc3 if (x_lo <= junction_x(j) <= x_hi)]

# --- (2) Eliminar tubos completamente a un lado y reindexar todo lo que dependa del índice del tubo ---
n_tubes = len(tubes)
keep_mask = np.ones(n_tubes, dtype=bool)
for k in range(n_tubes):
    if tube_both_sides_out(k):
        keep_mask[k] = False

old_to_new = {}
new_index = 0
for k in range(n_tubes):
    if keep_mask[k]:
        old_to_new[k] = new_index
        new_index += 1
# nuevo número de tubos:
n_tubes_kept = new_index

# Reempaquetamos arrays/listas indexadas por tubo
tubes_kept           = [tubes[k] for k in range(n_tubes) if keep_mask[k]]
flows_kept           = np.array([flows[k] for k in range(n_tubes) if keep_mask[k]], dtype=float)
oriented_tangents_kept = [oriented_tangents[k] for k in range(n_tubes) if keep_mask[k]]
real_midpoints_kept  = [real_midpoints[k] for k in range(n_tubes) if keep_mask[k]]
L_list_kept          = [L_list[k] for k in range(n_tubes) if keep_mask[k]] if 'L_list' in globals() else None

# --- Rehacer el índice "tube_index" (lista de tubos incidentes en cada nodo del grafo) con los índices nuevos ---
tube_index_kept = {}
for j,node_tubes in tube_index.items():
    kept = [old_to_new[k] for k in node_tubes if keep_mask[k]]
    if kept:
        tube_index_kept[j] = kept

# --- Actualizar junc3 → ya filtradas y con tubos reindexados (pueden pasar a tener 1,2 o 3 tubos) ---
juncs_after = []
for j in junc3_filtered:
    if j in tube_index_kept:
        juncs_after.append(j)
# (Si alguna junction filtrada pierde todos los tubos, se descarta.)

# --- (3) Añadir junctions de un solo tubo para extremos FUERA cuando el tubo cruza la ventana en x ---
#     Creamos IDs sintéticos para "junctions de corte" de la forma ("cut", old_k, side)
#     Nota: no hacen falta coordenadas nuevas para escribir el fichero de links; si las quieres,
#     podrías calcular la intersección con x=x_lo/x_hi. Aquí guardamos una coordenada aproximada
#     (el nodo exterior ya tiene posición en 'pos').
synthetic_juncs = []           # lista de IDs de junctions sintéticas
synthetic_pos   = {}           # ID -> (x,y)
synthetic_incidence = {}       # ID -> [tube_index (nuevo)]

def add_synthetic_for_endpoint(j_ext, new_k):
    """Añade una junction sintética 'de un tubo' para el extremo exterior j_ext del tubo new_k."""
    key = ("cut", new_k, j_ext)  # único por tubo y nodo exterior
    if key not in synthetic_pos:
        synthetic_pos[key] = pos[j_ext]  # coordenadas del nodo "exterior" original
        synthetic_incidence[key] = [new_k]
        synthetic_juncs.append(key)

# Detectar cruces y añadir junctions de un tubo
for old_k in range(n_tubes):
    if not keep_mask[old_k]:
        continue
    new_k = old_to_new[old_k]
    ja, jb = tube_end_junctions(old_k)
    xa, xb = junction_x(ja), junction_x(jb)
    inside_a = (x_lo <= xa <= x_hi)
    inside_b = (x_lo <= xb <= x_hi)
    if inside_a and inside_b:
        continue  # totalmente dentro
    if not inside_a and not inside_b:
        continue  # ambos fuera: ya descartado más arriba
    # exactamente uno fuera → añadimos junction sintética para el "fuera"
    if not inside_a:
        add_synthetic_for_endpoint(ja, new_k)
    if not inside_b:
        add_synthetic_for_endpoint(jb, new_k)

# --- Fusionar las junctions finales: reales (dentro de la ventana y con tubos) + sintéticas ---
#     Mantenemos un diccionario de incidencia (junction -> lista de tubos incidentes) con índices NUEVOS.
junction_to_tubes = {}

# Reales dentro (pueden tener 1,2,3 tubos tras el filtrado)
for j in juncs_after:
    junction_to_tubes[j] = list(tube_index_kept.get(j, []))

# Sintéticas (1 solo tubo)
for jid in synthetic_juncs:
    junction_to_tubes[jid] = list(synthetic_incidence[jid])

# --- (4) Construcción de links dirigidos según sentido del flujo y escritura de fichero ---
# Para cada tubo (índice nuevo), determinamos su junc_in (upstream) y junc_out (downstream)
# usando la tangente orientada en el midpoint.
# Luego calculamos los pesos normalizando por el caudal que SALE de cada junction origen.

# 4.1: Mapeo (tubo -> par de junctions finales que toca dentro de nuestro conjunto)
#      Para extremos fuera, usamos la junction sintética correspondiente si existiese.
from collections import defaultdict

def endpoint_junction_id(old_k, which):
    """Devuelve el ID de junction (real o sintética) en el extremo 'which' (0 para inicio, 1 para final)
       del tubo old_k, usando índices NUEVOS y ventana en x."""
    ja, jb = tube_end_junctions(old_k)
    j_ext = ja if which == 0 else jb
    xj = junction_x(j_ext)
    inside = (x_lo <= xj <= x_hi)
    new_k = old_to_new[old_k]
    if inside:
        # si está dentro y además está en la lista final de junctions:
        if j_ext in junction_to_tubes:
            return j_ext
        else:
            # podría no estar si perdió todos sus tubos salvo éste; la añadimos:
            junction_to_tubes.setdefault(j_ext, []).append(new_k)
            return j_ext
    else:
        # buscar/crear sintética
        key = ("cut", new_k, j_ext)
        if key not in junction_to_tubes:
            # si aún no existe (por si llegamos aquí por primera vez)
            synthetic_pos[key] = pos[j_ext]
            junction_to_tubes[key] = [new_k]
            synthetic_juncs.append(key)
        return key

# Construimos (para tubos mantenidos) los pares (jA, jB) finales
tube_end_junc_ids = [None]*n_tubes_kept  # lista de (j0, j1) por índice nuevo
for old_k in range(n_tubes):
    if not keep_mask[old_k]:
        continue
    new_k = old_to_new[old_k]
    j0 = endpoint_junction_id(old_k, 0)
    j1 = endpoint_junction_id(old_k, 1)
    tube_end_junc_ids[new_k] = (j0, j1)

# 4.2: Dirección (in/out) según tangente orientada en el midpoint
def directed_pair_for_tube(new_k):
    """Devuelve (j_in, j_out) para el tubo new_k según el sentido del flujo."""
    t = np.asarray(oriented_tangents_kept[new_k])
    mx, my = real_midpoints_kept[new_k]
    jA, jB = tube_end_junc_ids[new_k]
    # posiciones (x,y): junction real -> pos[j]; sintética -> synthetic_pos
    def jxy(jid):
        return pos[jid] if isinstance(jid, tuple) and len(jid)==2 else synthetic_pos[jid]
    Ax, Ay = jxy(jA)
    Bx, By = jxy(jB)
    vA = np.array([Ax - mx, Ay - my])
    vB = np.array([Bx - mx, By - my])
    # El endpoint con proyección POSITIVA sobre la tangente orientada está "downstream"
    dpA = t.dot(vA)
    dpB = t.dot(vB)
    if dpA < dpB:
        # A más negativo/menos positivo -> upstream, B -> downstream
        j_in, j_out = jA, jB
    else:
        j_in, j_out = jB, jA
    return j_in, j_out

links = []  # (j_in, j_out, Q)
for k_new in range(n_tubes_kept):
    j_in, j_out = directed_pair_for_tube(k_new)
    links.append((j_in, j_out, float(flows_kept[k_new])))

# 4.3: Pesos normalizando por el caudal que SALE de cada junction de entrada
sum_out = defaultdict(float)
for j_in, j_out, q in links:
    sum_out[j_in] += q

# Asignamos IDs enteros a todas las junctions finales para escribir el fichero
all_junction_ids = list(junction_to_tubes.keys())
# Aseguramos un orden estable: primero reales (tuplas (i,j)), luego sintéticas ("cut", ...)
reals      = [jid for jid in all_junction_ids if isinstance(jid, tuple) and len(jid)==2]
synthetics = [jid for jid in all_junction_ids if not (isinstance(jid, tuple) and len(jid)==2)]
ordered_junctions = reals + synthetics

jid_to_int = {jid: idx for idx, jid in enumerate(ordered_junctions)}

# Escribimos fichero: junc_in, junc_out, weight
with open("junction_links.txt", "w") as f:
    for j_in, j_out, q in links:
        denom = sum_out[j_in]
        w = (q/denom) if denom > 0 else 0.0
        f.write(f"{jid_to_int[j_in]} {jid_to_int[j_out]} {w:.4f}\n")


print(f"[OK] Filtrado de junctions/tubos en x∈[{x_lo},{x_hi}]")
print(f"[OK] Tubos mantenidos: {n_tubes_kept} / {n_tubes}")
print(f"[OK] Junctions reales finales: {len(reals)}, sintéticas: {len(synthetics)}, total: {len(ordered_junctions)}")
print("[OK] Fichero 'junction_links.txt' guardado con líneas: junc_in, junc_out, weight")
# ==== FIN DEL BLOQUE ====


In [ ]:
# ==== GUARDAR COORDENADAS DE LAS JUNCTIONS FINALES ====
# Requisitos: ordered_junctions, jid_to_int, pos, synthetic_pos ya definidos.

# Preparamos una lista para guardar las coordenadas ordenadas por ID entero
# El tamaño es el número total de junctions finales
num_final_junctions = len(ordered_junctions)
coords_list = [None] * num_final_junctions

# Llenamos la lista usando el mapeo jid_to_int para el índice correcto
for jid, int_id in jid_to_int.items():
    if isinstance(jid, tuple) and len(jid) == 2:
        # Es una junction real, usamos el diccionario 'pos'
        coords_list[int_id] = pos[jid]
    else:
        # Es una junction sintética, usamos el diccionario 'synthetic_pos'
        coords_list[int_id] = synthetic_pos[jid]

# Escribimos el fichero: integer_id x_coordinate y_coordinate
output_coords_file = "junction_coordinates.txt"
with open(output_coords_file, "w") as f_coords:
    for int_id, (x, y) in enumerate(coords_list):
        f_coords.write(f"{int_id} {x:.6f} {y:.6f}\n") # Formato con 6 decimales

print(f"[OK] Coordenadas de las {num_final_junctions} junctions finales guardadas en '{output_coords_file}'")
# ==== FIN DEL BLOQUE DE GUARDAR COORDENADAS ====

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# WE PLOT THE MEDIUM DISPLAYING THE POISEUILLE PROFILES AND INDICATING WHICH TYPE EACH JUNCTION IS AND WICH ONE IS THE BIG TUBE IN EACH JUNCTION

# We rescale U vectors for better visibilityl
vel_mags = np.linalg.norm(Uall, axis=1)
max_vel = vel_mags.max() if vel_mags.max()>0 else 1.0

arrow_len = (xmax - xmin) * 0.002
vel_scale = 5*arrow_len / max_vel
U_scaled = Uall * vel_scale  

# we classify junctions
junc1, junc2 = [], []
for j in junc3:
    idxs = tube_index.get(j, [])
    if len(idxs) != 3:
        continue
    qs = np.array([flow_rates[i] for i in idxs])
    i_max = idxs[np.argmax(qs)]
    x0, y0 = real_midpoints[i_max]
    xj, yj = pos[j]
    dp = oriented_tangents[i_max].dot((xj-x0, yj-y0))
    if dp > 0:
        junc2.append(j)
    else:
        junc1.append(j)

# we plot tangent vectors
Xg, Yg, Ug, Vg = [], [], [], []
for k in range(len(tubes)):
    x0, y0 = real_midpoints[k]
    tx, ty = oriented_tangents[k]
    Xg.append(x0); Yg.append(y0)
    Ug.append(tx * arrow_len); Vg.append(ty * arrow_len)

# we display the biggest tube at each junction
Xy, Yy, Uy, Vy = [], [], [], []
for j in junc3:
    idxs = tube_index[j]
    qs = np.array([flow_rates[i] for i in idxs])
    i_max = idxs[np.argmax(qs)]
    xj, yj = pos[j]  # junction center
    tx, ty = oriented_tangents[i_max]
    Xy.append(xj); Yy.append(yj)
    Uy.append(tx * arrow_len); Vy.append(ty * arrow_len)

# GRAPH
fig, ax = plt.subplots(figsize=(12, 8), dpi=1200)
ax.imshow(mask, origin='lower',
          extent=(xmin, xmax, ymin, ymax),
          cmap='gray', alpha=0.4)

for k in range(len(tubes)):
    start, end = offsets[k], offsets[k+1]
    pts_k = all_pts[start:end]
    uscaled = U_scaled[start:end]
    xs, ys = pts_k[:,0], pts_k[:,1]
    
    ax.plot(xs, ys, color='black', linewidth=0.5, alpha=0.6, zorder=1)
    
    ax.quiver(xs, ys,
              uscaled[:,0], uscaled[:,1],
              angles='xy', scale_units='xy', scale=1,
              width=0.0002, color='cyan', alpha=0.8, zorder=2)

ax.quiver(
    Xg, Yg, Ug, Vg,
    angles='xy', scale_units='xy', scale=1,
    width=0.0004, headwidth=1, headlength=1.2, headaxislength=1,
    color='green', alpha=0.8, zorder=3
)

ax.quiver(
    Xy, Yy, Uy, Vy,
    angles='xy', scale_units='xy', scale=1,
    width=0.0004, headwidth=1, headlength=1.2, headaxislength=1,
    color='yellow', alpha=0.9, zorder=4
)

for j in junc1:
    xj, yj = pos[j]
    ax.text(xj, yj, "1", color='red', fontsize=2,
            fontweight='bold', ha='center', va='center', zorder=5)
for j in junc2:
    xj, yj = pos[j]
    ax.text(xj, yj, "2", color='blue', fontsize=2,
            fontweight='bold', ha='center', va='center', zorder=5)

ax.set_aspect('equal')
ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')
plt.tight_layout()
plt.show()